(tir_blitz)=
# TensorIR 的突击课程

**作者**: [Siyuan Feng](https://github.com/Hzfengsy)

TensorIR 是用于深度学习程序的特定域语言，有两个广泛的目的：

- 在各种硬件后端进行程序变换和优化的实现。
- 用于自动张量化程序优化的抽象。

In [1]:
import tvm
from tvm.script import ir as I
from tvm.ir.module import IRModule
from tvm.script import tir as T
import numpy as np

## IRModule

IRModule 是 TVM 的中心数据结构，它包含深度学习程序。它是 IR 变换和模型构建的基本关注对象。

```{image} https://daobook.github.io/tvm-web-data/images/design/tvm_life_of_irmodule.png
:width: 85%
```

这是 IRModule 的生命周期（life cycle），它可以从 TVMScript 创建。TensorIR 调度原语（primitive）和传递（pass）是变换 IRModule 的两种主要方式。另外，对 IRModule 进行一系列的变换也是可以接受的。请注意，可以在 **任何** 阶段向 TVMScript 打印 IRModule。在所有变换和优化完成后，可以将 IRModule 构建为可运行的模块，以部署在目标设备上。

基于 TensorIR 和 IRModule 的设计，能够创建新的编程方式：

1. 用 TVMScript 写基于 Python-AST 语法的程序。
2. 用 python api 变换和优化程序。
3. 通过命令式的变换 API，交互式地检查和尝试性能。

## 创建 IRModule

IRModule 可以通过编写 TVMScript 来创建，TVMScript 是 TVM IR 的可圆润化（round-trippable）的语法。

与通过 [张量表达式](tutorial-tensor-expr-get-started) 创建计算表达式不同，TensorIR 允许用户通过 TVMScript（嵌入式 python AST 的语言）来编程。这种新方法使得编写复杂的程序并进一步调度和优化它成为可能。

In [2]:
@I.ir_module
class MyModule:
    @T.prim_func
    def main(A: T.Buffer(8, "float32"), B: T.Buffer(8, "float32")):
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        for i in range(8):
            # block 是计算的抽象。
            with T.block("B"):
                # 定义 spatial block 迭代器，并将其绑定到值 i。
                vi = T.axis.spatial(8, i)
                B[vi] = A[vi] + 1.0


ir_module = MyModule
print(type(ir_module))
ir_module.show()

<class 'tvm.ir.module.IRModule'>


此外，还可以使用张量表达式 DSL 来编写简单的算子，并将其转换为 IRModule。

In [3]:
from tvm import te

A = te.placeholder((8,), dtype="float32", name="A")
B = te.compute((8,), lambda *i: A(*i) + 1.0, name="B")
func = te.create_prim_func([A, B])
ir_module_from_te = IRModule({"main": func})
ir_module_from_te.show()

## 构建和运行 IRModule

我们可以将 IRModule 构建为具有特定目标后端的可运行模块。

In [4]:
mod = tvm.build(ir_module, target="llvm")  # 用于 CPU 后端的模块
print(type(mod))

<class 'tvm.driver.build_module.OperatorModule'>


准备好输入 array 和输出 array，然后运行该模块。

In [5]:
a = tvm.nd.array(np.arange(8).astype("float32"))
b = tvm.nd.array(np.zeros((8,)).astype("float32"))
mod(a, b)
print(a)
print(b)

[0. 1. 2. 3. 4. 5. 6. 7.]
[1. 2. 3. 4. 5. 6. 7. 8.]


## 转换 IRModule

IRModule 是程序优化的中心数据结构，它可以通过 `Schedule` 进行转换。调度包含多个原语方法，以交互式地转换程序。每个原语都以某些方式改造程序，以带来额外的性能优化。

<img src="https://daobook.github.io/tvm-web-data/images/design/tvm_tensor_ir_opt_flow.png" align="center" width="100%">

上面的图片是优化张量程序的典型工作流程。首先，需要在由 TVMScript 或 Tensor Expression 创建的初始 IRModule 上创建调度。然后，一连串的调度原语将有助于提高性能。最后，我们可以将其降低并构建为可运行的模块。

这里只演示了非常简单的变换。首先，在输入的 `ir_module` 上创建调度。

In [6]:
sch = tvm.tir.Schedule(ir_module)
print(type(sch))

<class 'tvm.tir.schedule.schedule.Schedule'>


将该循环分为 3 个循环，并打印结果。

In [7]:
# 按 name 获取 block
block_b = sch.get_block("B")
# 获取 block 周围的 loops
(i,) = sch.get_loops(block_b)
# 平铺（tile）循环嵌套。
i_0, i_1, i_2 = sch.split(i, factors=[None, 2, 2])
sch.mod.show()

也可以重新调度循环的顺序。现在将循环 `i_2` 移到 `i_1` 的外面。

In [8]:
sch.reorder(i_0, i_2, i_1)
sch.mod.show()

### 转化为 GPU 程序

如果想在 GPU 上部署模型，线程绑定是必要的。幸运的是，也可以使用原语并做增量变换。

In [9]:
sch.bind(i_0, "blockIdx.x")
sch.bind(i_2, "threadIdx.x")
sch.mod.show()

绑定线程后，现在用 `cuda` 后端构建 IRModule。

In [ ]:
ctx = tvm.cuda(0)
cuda_mod = tvm.build(sch.mod, target="cuda")
cuda_a = tvm.nd.array(np.arange(8).astype("float32"), ctx)
cuda_b = tvm.nd.array(np.zeros((8,)).astype("float32"), ctx)
cuda_mod(cuda_a, cuda_b)
print(cuda_a)
print(cuda_b)